<a href="https://colab.research.google.com/github/SeidaAhmed/An-application-of-Machine-Learning-to-COVID-19-data/blob/main/TEAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
metrics_8k=[
"https://www.sec.gov/Archives/edgar/data/1650372/000165037225000005/ex991q2fy25.htm",
"https://www.sec.gov/Archives/edgar/data/1650372/000165037224000054/ex991q1fy25.htm",
"https://www.sec.gov/Archives/edgar/data/1650372/000165037224000023/ex991q4fy24.htm",
"https://www.sec.gov/Archives/edgar/data/1650372/000165037224000013/ex991q3fy24.htm",
"https://www.sec.gov/Archives/edgar/data/1650372/000165037224000007/ex991q2fy24.htm",
"https://www.sec.gov/Archives/edgar/data/1650372/000165037223000058/ex991q1fy24.htm",
"https://www.sec.gov/Archives/edgar/data/1650372/000165037223000035/ex991q4fy23.htm",
"https://www.sec.gov/Archives/edgar/data/1650372/000165037223000021/ex991q3fy23.htm",
#"https://www.sec.gov/Archives/edgar/data/1650372/000165037223000007/ex991q2fy23amendment.htm",
"https://www.sec.gov/Archives/edgar/data/1650372/000165037223000005/ex991q2fy23.htm",
"https://www.sec.gov/Archives/edgar/data/1650372/000165037222000075/ex991q1fy23.htm",
#"https://www.sec.gov/Archives/edgar/data/1650372/000165037222000071/investorpresentation1003.htm"
]

# metrics_8k=[
# "https://www.sec.gov/Archives/edgar/data/1650372/000165037225000005/ex991q2fy25.htm"]

Important imports

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import requests
from bs4 import BeautifulSoup
try:
    import urllib.request as urllib3
    #requests.packages.urllib3.contrib.pyopenssl.extract_from_urllib3()
except ImportError:
    import urllib3
import zlib
from nltk import Tree
import re
nltk.download('maxent_treebank_pos_tagger')
nltk.download('path')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

years = []
for i in range(2000,2100):
  years.append(i)
unit_conv={"billion":1000000000, "million":1000000,"millions":1000000,"thousand":1000,"%":0.01,"percentage":0.01,"percent":0.01,"":1,"basis points":0.0001,"bps":0.0001}


[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Error loading path: Package 'path' not found in index
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Functions that should generally stay the same:

In [ ]:
def extract_soup(url):

    hdr = {'User-Agent': 'UbineerCorp info@ubineer.com','Accept-Encoding': 'gzip, deflate','host': 'www.sec.gov'}
    s = requests.Session()
    s.headers.update(hdr)
    data = s.get(url)
    s.close()
    soup = BeautifulSoup(data.content, features="html.parser")
    return(soup)

def openBS(html_file):

  with open(html_file, encoding = "ISO-8859-1") as fp:

    soup = BeautifulSoup(fp,'html.parser')

  return soup

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def find_between_include(s, first, last, length):
    try:
        start = s.index( first ) - length
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""


def unitConversion(insert_lst):
  value = None
  insert_lst[0] = insert_lst[0].replace('flat','0%')
  unit_conv={"billion":1000000000,"million":1000000,"thousand":1000,"%":0.01,"percentage":0.01,"percent":0.01,"":1,"basis points":0.0001,"bps":0.0001}
  if len(insert_lst) != 2:

    for unit in list(unit_conv.keys()):
      if unit == '':
        pass
      elif unit in insert_lst[0]:
        insert_lst = [str(insert_lst[0]).replace(unit,''), unit]



  insert_lst[-1] =insert_lst[-1].lower()


  insert_lst[0] = str(insert_lst[0]).replace(',','').replace('$','').replace('(','-').replace(')','').replace('%','').strip()
  #Andi adds brackets


  insert_lst[0] = float(insert_lst[0])

  for ele in unit_conv.keys():
    #print(ele)
    if ele == insert_lst[-1]:
      value = insert_lst[0] * unit_conv[ele]
      #attempt to fix floating point error
      value = round(value,4)
    else:
      pass
  if value == None:
    print("Value is None: ", insert_lst)
  return value

def change_pos_tag(tag_sentence,model):
    new_tag_lst=[]
    for ele in tag_sentence:
        if ele[0] in model.keys():
            new_tag_lst.append((ele[0],model[ele[0]]))
        else:
            new_tag_lst.append(ele)
    return(new_tag_lst)

def listofdict (tree):
  # source: https://stackoverflow.com/questions/63945863/converting-nltk-chunks-to-a-list-of-dictionaries
  # need helper function to simply put chunks into dictionaries
    dlist = []
    d = {}
    for item in tree:

        if isinstance(item, Tree):

            d[item.label()] = ' '.join([l[0] for l in item.leaves()])
            dlist.append(d) if len(d)>0 else None
            d = dict()

    dlist.append(d) if len(d)>0 else None
    return dlist

def find_segment(listofdict):
    # helper function
    tl = "Timeline"
    tl_count = 0
    segment = 0
    for i in range(len(listofdict)):
        if tl in listofdict[i].keys():
            tl_count +=1
            if tl_count == 2:
                segment = i
                return segment
    return len(listofdict)

def combine_dict(listofdict):
  # merge messy dictionaries from stack overflow function
    master_outlook = []

    while(len(listofdict) > 0 ):
        segment = find_segment(listofdict)

        time_dict = {}
        for i in range(segment):
            time_dict.update(listofdict[i])

        listofdict = listofdict[segment:]
        master_outlook.append(time_dict)
    return master_outlook

Capture guidance function: This function captures all the text in the guidance section of the html file. It should generally stay the same unless the company has a weird format or you need to change out some specific words.

In [ ]:
def captureGuidance(eightK_lst):
    guidance_bs4_lst=[]
    for ele in eightK_lst:

        bs4_8K=extract_soup(ele)
        #print(bs4_8K.prettify())
        test= bs4_8K.get_text()
        #print(len(test))
        result = find_between( test,"Business Highlights" , "Financial Targets" )
        print(result)
        # if not result: # some companies call outlook guidance
        #   result = find_between( test, "Guidance", "Call")
        # elif not result: # pltr is inbetween outlook and earnings webcast
        #   result = find_between( test, "Outlook", "Earnings")


        #print(bs4_8K.select(test.name+':contains("Financial Outlook")'))
        #print(result)
        #print(bs4_8K.find(text="Quarterly Conference Call ").previous_element.parent)

        result = re.sub(r'(\d+)-(\d+)', r'\1 \2', result)
        result = re.sub(r'(\$\d+\.\d+)', r'\1 ', result)
        result = re.sub(r'\b(low|mid|high|single|teens|to)-\b', r'\1 ', result)

        replace = {'%':' % ','shares':' shares ','.For':' for','outstandingfull':' outstanding full ','20232exhibit':'2023 ','year over year':'year-over-year','millionfor':' million for ','\n':' ',
                '•': ' ','\xa028':' ','\xa017':' ','    ':'','◦':' ','million.':' million . ', '\xa0':' ','%.': ' % .','\uf0b7':' ','oRevenue':'revenue','oNon-GAAP':'non-gaap','●':' ','.Non-GAAP':'. non-gaap','.Revenues':'. revenues','breakeven':'$0'}
        for word in replace:
          result = result.replace(word,replace[word])
        guidance_bs4_lst.append(result.strip().lower())
    return guidance_bs4_lst

metrics_data = captureGuidance(metrics_8k)
metrics_data


:•A Leader in The Forrester Wave™: Knowledge Management Solutions, Q4 2024: Atlassian was named a Leader in The Forrester Wave for Knowledge Management Solutions, Q4 20241, the first-ever Wave for this market. Atlassian offers products like Confluence, Loom, and Rovo which allow all teams to unlock knowledge across their organization and transform traditional silos into enterprise-wide collaboration with an AI-first strategy.•A Leader in the 2024 Gartner® Magic Quadrant™ for Marketing Work Management Platforms: Atlassian was named a Leader in the 2024 Gartner Magic Quadrant for Marketing Work Management Platforms2. Atlassian tools like Jira and Rovo, along with deep integrations across a broad network of third-party applications enable marketing teams to plan and track work, align on goals, and reduce costs and context switching. •A Leader in the IDC MarketScape: Worldwide IT Service Management Software 2024 Vendor Assessment: Atlassian was named a Leader in the IDC MarketScape: Worldw

[': a leader in the forrester wave™: knowledge management solutions, q4 2024: atlassian was named a leader in the forrester wave for knowledge management solutions, q4 20241, the first-ever wave for this market. atlassian offers products like confluence, loom, and rovo which allow all teams to unlock knowledge across their organization and transform traditional silos into enterprise-wide collaboration with an ai-first strategy. a leader in the 2024 gartner® magic quadrant™ for marketing work management platforms: atlassian was named a leader in the 2024 gartner magic quadrant for marketing work management platforms2. atlassian tools like jira and rovo, along with deep integrations across a broad network of third-party applications enable marketing teams to plan and track work, align on goals, and reduce costs and context switching.  a leader in the idc marketscape: worldwide it service management software 2024 vendor assessment: atlassian was named a leader in the idc marketscape: worl

These are the custom chunking model words. This will vary depending on the company, but the core words will stay in here.

In [ ]:
model = {'first': 'CD', 'second': 'CD', 'third': 'CD', 'fourth': 'CD', 'total': 'CD', 'expected': 'expected', 'quarter': 'TIME', 'year': 'TIME', 'breakeven': 'CD', 'full':
'CD', 'tax': 'tax', 'subscription': 'sub', 'billings': 'bill', 'billion': 'unit', 'revenue': 'rev', 'unit': 'unit', 'full-year': 'TIME', 'fiscal': 'TIME', 'million': 'unit',
'share': 'share', 'shares': 'share', 'year-over-year': 'yoy', '%': '%', 'ebitda': 'ebitda', 'loss': 'loss', 'arr': 'arr', 'income': 'income', 'net': 'net', 'revenues': 'rev',
'non-gaap': 'non-gaap', 'gaap': 'gaap', 'operating': 'op', 'margin': 'mar', 'services': 'ser', 'adjusted': 'adj', 'negative': 'negative', 'respectively': 'resp', 'basic': 'basic',
'q1': 'TIME', 'q2': 'TIME', 'q3': 'TIME', 'q4': 'TIME', 'fy': 'TIME', 'expenses': 'expenses', 'quarter-over-quarter': 'qoq', 'expectation ': 'expect', 'gross': 'gross', 'growth': 'growth',
'rate': 'rate', 'operations': 'op', 'operation': 'op', 'cash': 'cash', 'expenditures': 'expenses', 'provision': 'prov', 'thousand': 'unit', 'earning': 'earn', 'taxes': 'tax', 'expense': 'expenses',
'interest': 'interest', 'subscriptions': 'sub', 'eps': 'eps', 'compensation': 'comp', 'amortization': 'amort', 'acquisition': 'acq', 'costs': 'cost', 'debt': 'debt', 'assumed': 'ass', 'common': 'com',
'dec.': 'month', 'break-even': 'CD', 'free': 'free', 'flow': 'flow', 'annual': 'ann', 'cloud': 'cloud', 'positive': 'pos', 'zero': 'CD', 'comparable': 'comp', 'comp': 'comp', 'return': 'rtn', 'invested': 'inv',
'capital': 'cap', 'dividend': 'div', 'payments': 'pay', 'target': 'target', 'consolidated': 'con',
'repurchases': 'repur', 'repurchase': 'repur','repurchased': 'repur', 'repayments': 'repay', 'maturities': 'matur', 'reduced': 'reduce', 'reducing': 'reduce', 'planned': 'plan', 'percent': '%', 'investments': 'inv',
'profit': 'profit', 'low-teens': 'CD', 'fromsource': 'src', 'organic': 'organic', 'expansion': 'growth', 'bps': 'bps', 'increase': 'growth', 'half': 'CD', 'back': 'CD', 'ending': 'end', 'ended': 'end', 'count': 'count',
'even': 'CD', 'single': '%', 'percentage': '%', 'low': 'CD', 'mid': 'CD', 'high': 'CD', 'flat': 'CD', 'teens': '%', 'margins': 'mar', 'core': 'core', 'non-u.s.': 'non-gaap', 'outlook': 'outlook', 'â€”': ':', 'homes': 'home',
'closed': 'closed', 'end': 'end', 'lower': 'reduce', 'sg': 'sg', 'pre-tax': 'pre-tax', 'increasing': 'growth', 'financial': 'financial', 'roic': 'roic', 'basis': 'basis', 'points': 'points', 'sales': 'sales', 'period': 'period',
 'double-digit': 'CD', 'saas': 'saas', 'license': 'license', 'attributable': 'att', 'stockholders': 'stkhdr', 'hardware': 'hardware', 'reduce': 'reduce', 'stock': 'stk', 'restoration': 'restoration', 'continuing': 'continuing',
 'intangibles': 'intan', 'non-cash': 'non-cash', 'stock-based': 'stk', 'service': 'ser', 'expectations': 'expect', 'aggregates': 'aggr', 'freight-adjusted': 'frtadj', 'price': 'price',
'digit': '%', 'asphalt': 'asphalt', 'concrete': 'concrete', 'calcium': 'calcium', 'sag': 'sag', 'depreciation': 'dpr', 'depletion': 'dpl', 'accretion': 'accr', 'shipment': 'spmt', 'cost': 'cost',
'down': 'negative', 'up': 'pos', 'volume': 'vol', 'tons': 'tons', 'spending': 'spend', 'pricing': 'price', 'non-aggregates': 'non-aggr', 'improvement': 'growth', 'improve': 'growth', 'diluted': 'dil', 'earnings': 'earn',
'foreign':'foreign', 'finance':'fin','currency':'currency','expect':'expect','marketable':'marketable','securities':'securities','headcount':'headcount'}

model.update({'january': 'month', 'february': 'month', 'march': 'month', 'april': 'month', 'may': 'month', 'june': 'month','july': 'month', 'august': 'month', 'september': 'month', 'october': 'month', 'november': 'month', 'december': 'month'})
#meta specific
model.update({'ad':'ad','dap':'dap','daus':'daus','map':'map','maus':'maus','impressions':'impr'})

Filter sentence function: Will need to change the stop words depending on the context.

In [ ]:
def filter_sent(sentence, model):
    custom_stop = ["approximately","/", ',','information','available',"'s",'follows',':','range','outstanding','respectively','average','basic','weighted-average','representing','weighted','ending','roughly','~','assuming', 'including', 'principal','payments','leases','projected']
    stop_words = set(stopwords.words("english"))
    filtered_sentence = []
    for w in sentence:
        if (w not in stop_words and w not in custom_stop) or (w in model.keys()):
            filtered_sentence.append(w)
    return filtered_sentence

Chunking function: This is the most difficult function to write. Completely depends on the way the company writes their sentences. When writing this function it's best to print(tagged) to see what's being shown. During this stage changes may need to be made to the model and custom stop words, as well as the captureGuidance function.

In [ ]:
def metrics_CHUNK(sentence, model):

    tagged_raw = nltk.pos_tag(sentence)

    tagged = change_pos_tag(tagged_raw,model) #https://www.guru99.com/pos-tagging-chunking-nltk.html
    print(tagged)
    chunk_rev = r"""
                    Timeline: {<CD><TIME>?<TIME><TIME><CD>}
                    Cust Cloud 10KARR: {<CD>}<NNS><JJR>
                    Total Customer Count:<basis>{<CD>}<NNS>


                """
    chunkParserrev = nltk.RegexpParser(chunk_rev)
    chunkedrev = chunkParserrev.parse(tagged)


    return listofdict(chunkedrev)

Outlook function: I don't think this ever changes but I have only done one text company so far so I am including it here: This should end up giving a list of dictionaries that grouped together correctly. Make sure to check that the output is correct.

In [ ]:

def metrics_info(data):
    Total_Outlook = []
    for i in range(len(data)):
        if data[i] != '':
            list_dict_chunks = metrics_CHUNK(filter_sent(word_tokenize((data[i])),model),model)
            Total_Outlook.extend(combine_dict(list_dict_chunks))

    return Total_Outlook

raw_info = metrics_info(metrics_data)
raw_info



[('leader', 'NN'), ('forrester', 'NN'), ('wave™', 'VBD'), ('knowledge', 'JJ'), ('management', 'NN'), ('solutions', 'NNS'), ('q4', 'TIME'), ('2024', 'CD'), ('atlassian', 'NN'), ('named', 'VBN'), ('leader', 'NN'), ('forrester', 'NN'), ('wave', 'VBP'), ('knowledge', 'VBN'), ('management', 'NN'), ('solutions', 'NNS'), ('q4', 'TIME'), ('20241', 'CD'), ('first-ever', 'JJ'), ('wave', 'NN'), ('market', 'NN'), ('.', '.'), ('atlassian', 'JJ'), ('offers', 'VBZ'), ('products', 'NNS'), ('like', 'IN'), ('confluence', 'NN'), ('loom', 'NN'), ('rovo', 'NN'), ('allow', 'JJ'), ('teams', 'NNS'), ('unlock', 'VBP'), ('knowledge', 'NN'), ('across', 'IN'), ('organization', 'NN'), ('transform', 'NN'), ('traditional', 'JJ'), ('silos', 'JJ'), ('enterprise-wide', 'JJ'), ('collaboration', 'NN'), ('ai-first', 'JJ'), ('strategy', 'NN'), ('.', '.'), ('leader', 'NN'), ('2024', 'CD'), ('gartner®', 'NN'), ('magic', 'NN'), ('quadrant™', 'NN'), ('marketing', 'NN'), ('work', 'NN'), ('management', 'NN'), ('platforms', 'NNS'

[{'Timeline': 'second quarter fiscal year 2025',
  'Cust Cloud 10KARR': '49,449'},
 {'Timeline': 'first quarter fiscal year 2025', 'Cust Cloud 10KARR': '46,844'},
 {'Timeline': 'fourth quarter fiscal year 2024',
  'Cust Cloud 10KARR': '45,842'},
 {'Timeline': 'third quarter fiscal year 2024', 'Cust Cloud 10KARR': '44,336'},
 {'Timeline': 'second quarter fiscal year 2024',
  'Cust Cloud 10KARR': '42,864'},
 {'Timeline': 'first quarter fiscal year 2024', 'Cust Cloud 10KARR': '40,103'},
 {'Timeline': 'third quarter fiscal year 2023',
  'Total Customer Count': '259,775'},
 {'Timeline': 'second quarter fiscal year 2023',
  'Total Customer Count': '253,177'},
 {'Timeline': 'first quarter fiscal year 2023',
  'Total Customer Count': '249,173'}]

Process function: Once all the data is in the correct dictionary, it needs to go through this process function to extract specifically the numbers. Uses the unitConversion function that's defined higher up

In [ ]:
def process(raw_info_dict): #If you ever find an error with this function let @Maxime know ASAP
    temp_dict = {}
    if "Drop" in raw_info_dict.keys():
      raw_info_dict.pop("Drop")

    if len(raw_info_dict)>1:
        temp_dict = {}
        temp_dict["Timeline"] = raw_info_dict["Timeline"]

        for key, value in raw_info_dict.items():
          value = value.replace("low teens", "11 % 13 %")
          value = value.replace("high teens","17 % 19 %" )
          value = value.replace("mid teens", "14 % 16 %")
          value = re.sub(r'(\d+)-(\d+)', r'\1 \2', value)#remove dash between two numbers
          arr = value.split()
          prefix = 1
          vals = []
          if (len(arr) == 0 or key=="Timeline"):
            continue
          conversions = [] #case where there are multiple conversions in the same captured data
          conversion = ''
          #Find unit conversion
          for i, elem in enumerate(arr):
            if elem == 'breakeven' or elem == 'break-even' or elem == 'zero': #replace words meaning 0
              arr[i] = '0'
            if elem in unit_conv.keys():
              conversion = elem
              conversions.append(conversion)
          # Check if unit conversion is in key
          if conversion == '':
            for word in key.split():
              if word.replace("(","").replace(")","") in unit_conv.keys(): #Replace unit conversion in key, ie. if key is "Revenue (in millions)", it should be "Revenue"
                conversion = word.replace("(","").replace(")","")
                conversions.append(conversion)

          if len(conversions) == 0:
            conversions.append('')

          if len(set(conversions)) > 1:
            multi = True
          else:
            multi = False

          # regex finds () and [] brackets
          key = re.sub("\(.*?\)|\[.*?\]","", key)
          #Check if numbers should be negative
          if ("loss" in arr) or ("negative" in arr) or ("decrease" in arr):
            prefix = -1
          conv_num = 0
          #Find values (should be 1 or 2)


          # IMPROVED HANDLING OF NEGATIVE AND POSITIVE VALS IN SAME CHUNK - by Jack Li
          ignore_set = ['$']
          switch = False
          negative = False
          state = 0
          for elem in arr:
            if "(" in elem and state == 0:
              state = 1
            if (elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').isdigit() and state == 1) or (elem == 'loss' and state == 1):
              state = 2
            if ")" in elem and state == 1:
              state = 0
            if ")" in elem and state == 2:
              switch = True
              break

          pos_search = False

          if switch is False:
            for elem in arr:
              if elem == 'positive':
                pos_search = True
                continue
              if pos_search:
                if (not elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit()) and (elem not in ignore_set):
                  pos_search = False

              if elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit():
                if pos_search:
                  vals.append(unitConversion([elem, conversions[conv_num]]))
                  if multi: conv_num += 1
                  pos_search = False
                else:
                  vals.append(unitConversion([elem, conversions[conv_num]])*prefix)
                  if multi: conv_num += 1
          else:
            for elem in arr:
              if "(" in elem:
                negative = True
              if ")" in elem:
                negative = False
              if elem.replace(".","").replace(",","").replace("$","").replace('(','').replace(')','').replace('%','').replace('-','').replace('~','').isdigit():
                if (negative):
                  vals.append(unitConversion([elem, conversions[conv_num]])*-1)
                  if multi: conv_num += 1
                else:
                  vals.append(unitConversion([elem, conversions[conv_num]]))
                  if multi: conv_num += 1

          if max(vals)-min(vals) > 100 * unit_conv[conversion]:
            for val in vals:
              if val/unit_conv[conversion] in years:
                  vals.remove(val)
                  break

          temp_dict[key] = max(vals)
          #temp_dict[key+" Lower"] = min(vals)

    return temp_dict

Clean function calls process on each dictionary in the list and returns a list of dictionaries where just the value is shown

In [ ]:
def clean(raw_info):
    Outlook = []
    for i in range(len(raw_info)):
        if process(raw_info[i]) != None:
            Outlook.append(process(raw_info[i]))
    return Outlook

data = clean(raw_info)

cleaned_data = [i for i in data if len(i)>1]
cleaned_data

[{'Timeline': 'second quarter fiscal year 2025', 'Cust Cloud 10KARR': 49449.0},
 {'Timeline': 'first quarter fiscal year 2025', 'Cust Cloud 10KARR': 46844.0},
 {'Timeline': 'fourth quarter fiscal year 2024', 'Cust Cloud 10KARR': 45842.0},
 {'Timeline': 'third quarter fiscal year 2024', 'Cust Cloud 10KARR': 44336.0},
 {'Timeline': 'second quarter fiscal year 2024', 'Cust Cloud 10KARR': 42864.0},
 {'Timeline': 'first quarter fiscal year 2024', 'Cust Cloud 10KARR': 40103.0},
 {'Timeline': 'third quarter fiscal year 2023',
  'Total Customer Count': 259775.0},
 {'Timeline': 'second quarter fiscal year 2023',
  'Total Customer Count': 253177.0},
 {'Timeline': 'first quarter fiscal year 2023',
  'Total Customer Count': 249173.0}]